In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Combined_red_flags_19_2_2020.csv')

In [3]:
from sklearn.utils import shuffle
df = shuffle(df)

In [4]:
df.head()

,selects,distincts,distinct values,red flags,query
1119,Quantity,Category Level 1,Transportation and Storage and Mail Services,Transportation and Storage and Mail Services h...,NaN
281,Payment,Category Level 1,Healthcare Services,Industrial Cleaning Services has the maximum g...,NaN
950,Delayed PO Payment,Category Level 1,Healthcare Services,Healthcare Services has 3380 times more Delaye...,NaN
506,Paid Amount,Category Level 1,Structures and Building and Construction and M...,Structures and Building and Construction and M...,NaN
434,Delayed PO Payment,Category Level 1,Food Beverage and Tobacco Products,21 Category Level 1 Engineering and Research a...,NaN


In [5]:
df.to_csv('combined_red_flags_20_2_2020.csv', index=False)

In [2]:
df = pd.read_csv('red_flags_orm_templates.csv')

In [8]:
df['Analysis Type']

0     Time Comparison
1     Time Comparison
2     Time Comparison
3     Time Comparison
4     Time Comparison
           ...       
72       Contribution
73       Contribution
74       Contribution
75         Comparison
76         Comparison
Name: Analysis Type, Length: 77, dtype: object

In [13]:
df['ORM Structure'].loc[(df['Analysis Type'] == 'Time Comparison') & (df['Statement Contains'] == 'compared to the last year')].iloc[0]

'{"table_id":"","operators_prefix":"percentage_change_in","operators_suffix":"yearly","measures":[],"by_dimension_values":[],"in_dimension_values":[],"in_time_dimension_values":[],"by_time_dimension_values":[],"conditionals":[],"others":[],"tablename":"","database":""}'

In [1]:
#imports
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
import os
import ast
import pymysql
import itertools
from clickhouse_driver import Client
from queries_for_insights import *
import traceback

# path = os.path.join(os.path.dirname(__file__))

host = "sia.ch01hqj8p0kh.ap-south-1.rds.amazonaws.com"
dbname = "kaggle_procurement"
user = "sia_mumbai_dev"
password = "Forty4Hz$admin2020"
conn = pymysql.connect(host, user=user, passwd=password, db=dbname)
df = pd.read_sql('SELECT * FROM data_lookup', con=conn)
select_df = pd.read_sql('SELECT * FROM select_lookup', con=conn)

clickhouse_client = Client(host='13.235.187.249', user='forty4hz_ro', password='Forty4Hz_Ro@2020')

vendor_invoice = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/vendor_invoice_dbeaver_03_02_20.csv')
contract = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/contract_dbeaver_03_02_20.csv')
pr_to_po = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/pr_to_po_dbeaver_03_02_20.csv')
source_to_contract = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/source_to_contract_dbeaver_03_02_20.csv')

tables_map = {
    1:contract,
    2:pr_to_po,
    3:source_to_contract,
    5:vendor_invoice
}

tables_string_map = {
    1:'contract',
    2:'pr_to_po',
    3:'source_to_contract',
    5:'vendor_invoice'
}

In [14]:
vendor_invoice.groupby(by=['Item']).agg({'Vendor':'count'})

,Vendor
Item,
! year Repair Service Advantage,1
"""0L3LWZZ1-ER1GS - Symc Backup Exec 2014 Server Win Per Server Renewal Essential",1
"""12RU Mobile rack equipped with locking front and rear doors. Top grommet Acryli",1
"""A.O. SMITH"" MOTOR 1/4 HP, 115V, 5.4 A",1
"""Advice for New Parents Guide"" Print Job",1
...,...
zuchini 40#(d91)50446229,1
zuchini40#(d91)50446229,1
zurn NHZN or equal,1


In [19]:
len(vendor_invoice['Vendor'][vendor_invoice['Item'] == 'IT Consulting Services'].drop_duplicates())

64

In [5]:
gb_object = vendor_invoice.groupby(by=['Item']).agg({'Vendor':'count'}).sort_values(by=['Vendor'], ascending=False)

In [22]:
gb_object.sort_values(by=['Vendor'], ascending=False)

,Vendor
Item,
Medical Supplies,2916
Contract,2092
ew,1539
Expert Witness,1317
medical vocational training,1092
...,...
Whole Food Juice Tubes,0
POULTRY LOG 2/9 - 10 LBS PER CASE,0
HP Tipping Point,0


In [6]:
from tqdm import tqdm

In [8]:
for i in tqdm(gb_object.index.values):
    print(i)
    x = (len(np.unique(vendor_invoice['Vendor'][vendor_invoice['Item'] == i])))

  0%|                                                                             | 4/180025 [00:00<1:32:57, 32.27it/s]

Medical Supplies
Contract
ew
Expert Witness
medical vocational training
Toner
contract
Office Supplies


  0%|                                                                            | 12/180025 [00:00<1:29:32, 33.51it/s]

Dental Supplies
toner
Service Contract
Vocational Rehabilitation Services
Low Income Home Energy Assistance Program
medical supplies
tra
medical vocational training services


  0%|                                                                            | 20/180025 [00:00<1:25:38, 35.03it/s]

...

  0%|                                                                           | 158/180025 [00:05<1:56:15, 25.78it/s]

cereal
Local Assistance for HIV/AIDS Services
Amendment No 1
Pest Control Services
Consulting Services
CHICKEN
emulsion


  0%|                                                                           | 161/180025 [00:05<1:40:15, 29.90it/s]

Copier
beef
coffee
C.O. 003
IT Consulting Services


TypeError: '<' not supported between instances of 'str' and 'float'

In [2]:
kyd_year = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/kyd-combinations-year.csv')
print(kyd_year.shape)
kyd_year = kyd_year.sort_values(by=['no_of_outliers'], axis=0)
# kyd_year = kyd_year[:50]
kyd_year = kyd_year[kyd_year['no_of_outliers'] > 800]
print(kyd_year.shape)
kyd_quarter = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/kyd-combinations-quarter.csv')
kyd_month = pd.read_csv('C:/Users/karthik/Documents/GitHub/Forty4Hz_Flask/red_flags/kyd-combinations-month.csv')


(12502, 15)
(104, 15)


In [7]:
def trending_analysis(Analysis_Input):

    keys = []
    values = []
    by_dimension_query_part = []

    for key, value in Analysis_Input['dimension'].items():
        
        keys.append(key)
        values.append(value)
    
        for sub_value in value:
            if not value.index(sub_value) == len(value)-1:
                by_dimension_query_part.append(f"{key} = '{sub_value}'")
            else:
                by_dimension_query_part.append(f"{key} = '{sub_value}'")
    
    by_dimension_query_part = " and ".join([i for i in by_dimension_query_part])
    by_dimension_query_part = "and "+by_dimension_query_part

    #handling the measure part
    
    formula = select_df.loc[select_df['Select'] == Analysis_Input['measure'][0], 'Formula'].iloc[0]
    table_select_belongs_to = select_df.loc[select_df['Select'] == Analysis_Input['measure'][0], 'Table_Select_Belongs_To'].iloc[0]
    time_dimension = select_df.loc[select_df['Select'] == Analysis_Input['measure'][0], 'Time_Dimension_1'].iloc[0]
    string_table = tables_string_map[int(table_select_belongs_to)]
    mapped_table = tables_map[int(table_select_belongs_to)]
    print(mapped_table[time_dimension])
    copy_of_mapped_table = mapped_table

#     mapped_table[time_dimension] = (pd.to_datetime(mapped_table[time_dimension]))
#     latest_year = max(pd.DatetimeIndex(mapped_table[time_dimension]).year.tolist())
#     mapped_table = mapped_table[mapped_table[time_dimension].dt.year == latest_year]
#     latest_month = (max(mapped_table[time_dimension].dt.month))
#     latest_quarter = (max(mapped_table[time_dimension].dt.quarter))
    
#     print(latest_year)
#     print(latest_quarter)
#     print(latest_month)
    sys.exit()


    if Analysis_Input['time_frame'][0] == 'year':
        
        #getting year, quarter, month values
        print(copy_of_mapped_table.shape)
        copy_of_mapped_table[time_dimension] = pd.to_datetime(mapped_table[time_dimension])
        year_values = mapped_table[time_dimension].dt.year
        quarter_values = mapped_table[time_dimension].dt.quarter
        month_values = mapped_table[time_dimension].dt.month
        
        copy_of_mapped_table['year'] = year_values
        copy_of_mapped_table['quarter'] = quarter_values
        copy_of_mapped_table['month'] = month_values
        
        year_groupby = copy_of_mapped_table.groupby(by=['year'], as_index=True).agg({formula:sum})
        print(year_groupby.head())



    elif Analysis_Input['time_frame'] == 'quarter':
        pass



    elif Analysis_Input['time_frame'] == 'month':
        pass

In [8]:
for measure, dim, dim_val in (zip(kyd_year['select'].values, kyd_year['distinct'].values, kyd_year['by_dim'].values)):
    
    print(measure, 'is measure')
    print(dim, 'is dim')
    print(dim_val, 'is dim val')
    
    dummy_insights_input = {
        'measure':[measure],
        'dimension':{
            dim:[dim_val]
        },
        'time_frame':['year']
    }

    trending_analysis(dummy_insights_input)
    sys.exit()


Price is measure
Category_Level_1 is dim
Healthcare Services is dim val
0         2017-07-03
1         2017-10-22
2         2017-12-28
3         2017-11-09
4         2017-11-13
             ...    
345966    2020-04-25
345967    2020-03-13
345968    2020-04-17
345969    2020-03-21
345970    2020-05-17
Name: Invoice_Issue_Date, Length: 345971, dtype: object


SystemExit: 

C:\Users\karthik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
Analysis_Input = {
    'measure':['Price'],
    'dimension':{
        'Category_Level_1':['Healthcare Services']
    },
    'time_frame':['year']
}

In [26]:
formula = select_df.loc[select_df['Select'] == Analysis_Input['measure'][0], 'Formula'].iloc[0]
table_select_belongs_to = select_df.loc[select_df['Select'] == Analysis_Input['measure'][0], 'Table_Select_Belongs_To'].iloc[0]
time_dimension = select_df.loc[select_df['Select'] == Analysis_Input['measure'][0], 'Time_Dimension_1'].iloc[0]
string_table = tables_string_map[int(table_select_belongs_to)]
mapped_table = tables_map[int(table_select_belongs_to)]
copy_of_mapped_table = mapped_table
print(copy_of_mapped_table.shape)

(345971, 38)


In [24]:
copy_of_mapped_table['year'] = pd.DatetimeIndex(copy_of_mapped_table[time_dimension]).year
copy_of_mapped_table['quarter'] = pd.DatetimeIndex(copy_of_mapped_table[time_dimension]).quarter
copy_of_mapped_table['month'] = pd.DatetimeIndex(copy_of_mapped_table[time_dimension]).month

In [27]:
copy_of_mapped_table = copy_of_mapped_table[copy_of_mapped_table[dim] == dim_val]
print(copy_of_mapped_table.shape)

(7977, 38)


In [34]:
copy_of_mapped_table = copy_of_mapped_table.groupby(by=['year','month'], as_index=True).agg({formula:sum})

In [70]:
copy_of_mapped_table

Unit_Price_In_USD
year month                   
2017 6           2.377587e+08
     7           7.651349e+09
     8           1.357464e+09
     9           8.670937e+08
     10          1.509055e+09
     11          3.315610e+08
     12          1.136595e+08
2018 1           3.179953e+08
     2           8.255697e+08
     3           2.262921e+10
     4           8.044021e+09
     5           4.818663e+09
     6           3.681437e+09
     7           4.387682e+08
     8           3.453140e+09
     9           4.353835e+09
     10          1.153983e+09
     11          5.599472e+08
     12          1.817992e+08
2019 1           8.040049e+07
     2           3.692023e+09
     3           4.142154e+09
     4           2.448653e+09
     5           2.473136e+08
     6           9.804310e+09
     7           4.942865e+08
     8           1.307311e+09
     9           4.542002e+08
     10          7.864163e+08
     11          4.582435e+08
     12          8.440235e+08
2020 1           1.216044e+09
     2           2.202569e+09
     3           2.663800e+08
     4           9.687704e+09
     5           6.697867e+09
     6           4.290600e+09

In [69]:
copy_of_mapped_table.index.get_level_values(0)

Int64Index([2017, 2017, 2017, 2017, 2017, 2017, 2017, 2018, 2018, 2018, 2018,
            2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2019, 2019, 2019,
            2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2020, 2020,
            2020, 2020, 2020, 2020],
           dtype='int64', name='year')

In [22]:
import datetime
from sklearn.linear_model import LinearRegression

In [35]:
df = pd.DataFrame({'year':[0,2018,2019,2020], 'quarter':[0,1,2,3], 'month':[0,1,2,3],'value':[1,2,3,4]})
df

,year,quarter,month,value
0,0,0,0,1
1,2018,1,1,2
2,2019,2,2,3
3,2020,3,3,4


In [49]:
df[df.nunique(1).eq(2)]

,year,quarter,month,value
0,0,0,0,1


In [29]:
dates = []
for i in range(1,13):
    dates.append(datetime.datetime(2018,4,i))

In [32]:
first_month = pd.datetime(2020, 1, 1)
last_month = pd.datetime(2020, 10, 1)

In [33]:
pd.date_range(start=first_month, end=last_month, freq='MS').to_julian_date()

Float64Index([2458849.5, 2458880.5, 2458909.5, 2458940.5, 2458970.5, 2459001.5,
              2459031.5, 2459062.5, 2459093.5, 2459123.5],
             dtype='float64')

In [33]:
import requests, os, json

In [23]:
self_reference = "http://13.232.213.75:3200"

In [28]:
temp_orm_input = {
    "by_dimension_values": ["*by_Category_Level_1"],
    "by_time_dimension_values": [],
    "conditionals": [],
    "in_dimension_values": [],
    "in_time_dimension_values": [],
    "measures": ["~spend"],
    "operators_prefix": "",
    "operators_suffix": "",
    "others": [],
    "table_id": "5"
}

In [29]:
query = str(self_reference) + "/orm"
response = requests.post(query, json=temp_orm_input)
output = response.text

In [36]:
json_op = json.loads(output)

In [37]:
json_op['query']

'Select Category_Level_1 as "Category Level 1", SUM(Invoice_Amount_In_USD) as "Spend" from kaggle_procurement.vendor_invoice group by Category_Level_1 order by Category_Level_1'